# 1.导入数据集

In [2]:
import pandas as pd
import numpy as np

In [3]:
Train=pd.read_excel(r'./训练集.xlsx',index_col=[0,1])
Test=pd.read_excel(r'./测试集.xlsx',index_col=[0,1])

删除湖北武汉的数据

In [4]:
Train.drop(index=('湖北','武汉'),inplace=True)
Test.drop(index=('湖北','武汉'),inplace=True)

将迁入迁出求和和比例求和，作为2个X

In [6]:
New_train=pd.DataFrame(Train.iloc[:,0:14].sum(axis=1),columns=['迁徙指数(训练集)'])
New_train['武汉出来的比例(训练集)']=Train.iloc[:,14:21].sum(axis=1).values
New_train['新增感染人数']=Train.iloc[:,-1].values

In [10]:
New_train.corr()  # 相关系数分析 - 新增感染人数跟比例的相关性最大

,迁徙指数(训练集),武汉出来的比例(训练集),新增感染人数
迁徙指数(训练集),1.000000,0.106550,0.234939
武汉出来的比例(训练集),0.106550,1.000000,0.931785
新增感染人数,0.234939,0.931785,1.000000


In [11]:
New_test

迁徙指数(测试集)  武汉出来的比例(测试集)  新增感染人数(测试集)
省份  地区                                        
上海  上海       166.55        0.0334          116
云南  临沧         4.99        0.0000            0
    丽江         8.07        0.0000            1
    保山         7.49        0.0000            2
    大理州       15.02        0.0000           11
...             ...           ...          ...
黑龙江 绥化        15.96        0.0000           26
    鸡西         3.12        0.0000           25
    鹤岗         2.42        0.0000            1
    黑河         3.65        0.0000            1
    齐齐哈尔      11.51        0.0000           24

[368 rows x 3 columns]

In [12]:
New_test=pd.DataFrame(Test.iloc[:,0:14].sum(axis=1),columns=['迁徙指数(测试集)'])

New_test['武汉出来的比例(测试集)']=Test.iloc[:,14:21].sum(axis=1).values

New_test['新增感染人数(测试集)']=Test.iloc[:,-1].values

In [13]:
New_test.corr() 

,迁徙指数(测试集),武汉出来的比例(测试集),新增感染人数(测试集)
迁徙指数(测试集),1.000000,0.067783,0.145661
武汉出来的比例(测试集),0.067783,1.000000,0.965328
新增感染人数(测试集),0.145661,0.965328,1.000000


# 建模


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
sc_X = StandardScaler()
sc_Y = StandardScaler()

# 多层感知机

In [15]:
from sklearn.neural_network import MLPRegressor
mlp_model = MLPRegressor(verbose=True,
                        learning_rate_init=0.01,
                         max_iter=1000,
                         hidden_layer_sizes=(10,2),
                         alpha=0.001,
                         random_state=94
                        )

In [16]:
sc_X_stder = sc_X.fit_transform(New_train.iloc[:,[0,1]].values)
sc_Y_stder = sc_Y.fit_transform(New_train.iloc[:,2].values.reshape(-1,1))

In [17]:
y_test = New_test.iloc[:,2].values.reshape(-1,1) #y_test为测试集真实值

In [46]:
poly_feat = PolynomialFeatures()  # 多项式
poly_feat_X = poly_feat.fit_transform(sc_X_stder)

In [47]:
mlp_model.fit(poly_feat_X,sc_Y_stder.reshape(368))

Iteration 1, loss = 1.41258638
Iteration 2, loss = 1.36341845
Iteration 3, loss = 1.09844208
Iteration 4, loss = 0.86125380
Iteration 5, loss = 0.67762554
Iteration 6, loss = 0.54553734
Iteration 7, loss = 0.43788859
Iteration 8, loss = 0.35231764
Iteration 9, loss = 0.29377332
Iteration 10, loss = 0.26070614
Iteration 11, loss = 0.22026641
Iteration 12, loss = 0.20410705
Iteration 13, loss = 0.18333272
Iteration 14, loss = 0.16635383
Iteration 15, loss = 0.13704731
Iteration 16, loss = 0.12833599
Iteration 17, loss = 0.11314196
Iteration 18, loss = 0.11204897
Iteration 19, loss = 0.11463633
Iteration 20, loss = 0.11353036
Iteration 21, loss = 0.10580043
Iteration 22, loss = 0.09643032
Iteration 23, loss = 0.08858524
Iteration 24, loss = 0.08734913
Iteration 25, loss = 0.08315591
Iteration 26, loss = 0.07960752
Iteration 27, loss = 0.07643810
Iteration 28, loss = 0.07659734
Iteration 29, loss = 0.07562912
Iteration 30, loss = 0.07302143
Iteration 31, loss = 0.06947337
Iteration 32, los

MLPRegressor(activation='relu', alpha=0.001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=(10, 2), learning_rate='constant',
             learning_rate_init=0.01, max_iter=1000, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=94, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=True, warm_start=False)

In [48]:
mlp_model.score(poly_feat_X,sc_Y_stder) # MLP模型训练集上的R2

0.9119120262411091

In [49]:
 # 测试集预测
sc_X_stder_test = sc_X.transform(New_test.iloc[:,[0,1]].values) # 测试集X归一化

poly_feat_X_test = poly_feat.transform(sc_X_stder_test) # 多项式化

y_test_pred = mlp_model.predict(poly_feat_X_test) 

sc_Y_true = sc_Y.inverse_transform(y_test_pred) #将预测结果反归一化

from sklearn.metrics import r2_score
r2_score(y_test,sc_Y_true)  # 测试集上的R2

0.7352483160384201

# 多项线性回归

In [22]:
from sklearn.linear_model import LinearRegression as LR
from sklearn.preprocessing import PolynomialFeatures
lr_model = LR()

sc_X_stder = sc_X.fit_transform(New_train.iloc[:,[0,1]].values)
sc_Y_stder = sc_Y.fit_transform(New_train.iloc[:,2].values.reshape(-1,1))

poly_feat = PolynomialFeatures() 

poly_feat_X = poly_feat.fit_transform(sc_X_stder) # 多项式化

In [23]:
y_test = New_test.iloc[:,2].values.reshape(-1,1) #y_test 为测试集真实值

In [24]:
lr_model.fit(poly_feat_X,sc_Y_stder)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [25]:
lr_model.score(poly_feat_X,sc_Y_stder) # 训练集上的R2

0.8914934227244807

In [26]:
 # 测试集预测
sc_X_stder_test = sc_X.transform(New_test.iloc[:,[0,1]].values) # 归一化

poly_feat_X_test = poly_feat.transform(sc_X_stder_test) # 多项化

y_test_pred = lr_model.predict(poly_feat_X_test)

sc_Y_true = sc_Y.inverse_transform(y_test_pred) # 将预测结果反归一化

In [27]:
from sklearn.metrics import r2_score
r2_score(y_test,sc_Y_true) # 测试集R2

0.641249665958082

In [28]:
lr_model.coef_

array([[ 0.        ,  0.13575146,  1.076111  , -0.00576764,  0.02525897,
        -0.01937528]])